In [303]:
from models.arcs import load_alt_model_a, load_alt_model_b, load_inf_model, GenPhiloText
from utilities.loaders import load_file
from utilities.preprocessors import preprocess, map_value_to_index, init_sequences_a, init_sequences_b, decode_predictions
from utilities.visualizers import export_results

from tensorflow.keras.losses import CategoricalCrossentropy as cce_loss
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import CategoricalAccuracy, CategoricalCrossentropy as cce_metric

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import tensorflow as tf

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [229]:
corpus = load_file('./data/notes.txt')

In [230]:
corpus[:500]

'A simple idea “What is the meaning of life?”\nI asked as I learned through the works of Camus? \nOne step down, I felt a yearning of meaning in this world.\nIn this yearning I stumbled upon eastern philosophy;\nIkigai as the Japanese philosophers called it was a considerable way for me to find meaning at that certain point in my life. \nFollowed then another idea, a leap of faith as Kierkegaard would call it, yet I had no idea this was his idea. \nCalm followed after the storm, and then I took another'

In [231]:
len(corpus)

226750

In [232]:
chars = sorted(list(set(corpus)))
chars

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '´',
 'ç',
 'é',
 'ï',
 '–',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '…']

# Preprocessing corpus
* replace quotation marks like this '“'/'”' with this instead '"'
* replace single quotation marks like this '‘'/'’' with ''' instead
* replace this hyphen '–' with this hyphen '—'
* lowercase all words (for now)
* replace 3 consecutive '.' with  '…' instead

In [233]:
corpus = preprocess(corpus)
corpus[:500]

'a simple idea "what is the meaning of life?"\ni asked as i learned through the works of camus? \none step down, i felt a yearning of meaning in this world.\nin this yearning i stumbled upon eastern philosophy;\nikigai as the japanese philosophers called it was a considerable way for me to find meaning at that certain point in my life. \nfollowed then another idea, a leap of faith as kierkegaard would call it, yet i had no idea this was his idea. \ncalm followed after the storm, and then i took another'

In [234]:
len(corpus)

226961

In [235]:
chars = sorted(list(set(corpus)))
# chars = ['[UNK]'] + chars
chars

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '´',
 'ç',
 'é',
 'ï',
 '—',
 '…']

# Creating mapper from a unique character to its respective index

In [236]:
char_to_idx = map_value_to_index(chars)
idx_to_char = map_value_to_index(chars, inverted=True)

In [237]:
char_to_idx

In [238]:
idx_to_char

In [239]:
n_unique = len(char_to_idx.get_vocabulary())
n_unique

57

In [240]:
n_time_steps = 100
X, Y = init_sequences_a(corpus, char_to_idx, T_x=n_time_steps)
X

<tf.Tensor: shape=(226861, 100), dtype=int64, numpy=
array([[25,  2, 43, ..., 29,  2, 43],
       [ 2, 43, 33, ...,  2, 43, 44],
       [43, 33, 37, ..., 43, 44, 29],
       ...,
       [39, 42,  2, ..., 29, 36, 33],
       [42,  2, 44, ..., 36, 33, 29],
       [ 2, 44, 42, ..., 33, 29, 30]], dtype=int64)>

In [241]:
X.shape

TensorShape([226861, 100])

In [242]:
Y

<tf.Tensor: shape=(226861,), dtype=int64, numpy=array([44, 29, 40, ..., 29, 30, 11], dtype=int64)>

# convert Y data's indeces to their one hot vector representation

In [243]:
Y = tf.one_hot(Y, depth=n_unique)
Y

<tf.Tensor: shape=(226861, 57), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [244]:
Y[3]

<tf.Tensor: shape=(57,), dtype=float32, numpy=
array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)>

In [245]:
# number of examples
len(X)

226861

In [246]:
len(X[-1])

100

# Instantiate generative model A with set architecture

In [247]:
emb_dim = 32
n_a = 128

In [248]:
model = load_alt_model_a(n_unique=n_unique, T_x=n_time_steps, emb_dim=emb_dim, n_a=n_a)
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 100, 32)           1824      
                                                                 
 lstm_8 (LSTM)               (None, 100, 128)          82432     
                                                                 
 lstm_9 (LSTM)               (None, 128)               131584    
                                                                 
 dense_4 (Dense)             (None, 57)                7353      
                                                                 
 batch_normalization_4 (Bat  (None, 57)                228       
 chNormalization)                                                
                                                                 
 activation_13 (Activation)  (None, 57)                0         
                                                      

# Provide loss, optimizer, and metrics for both alternative models A and B

In [249]:
opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
loss = cce_loss()
metrics = [CategoricalAccuracy(), cce_metric()]

# model.compile(loss=loss, optimizer=opt, metrics=metrics)

# Train alternative model A and checkpoint weights

In [250]:
# weights_path = "./weights/weights-improvement-{epoch:02d}-{categorical_accuracy:.4f}.hdf5"
# checkpoint = ModelCheckpoint(weights_path, monitor='categorical_accuracy', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

In [251]:
# history = model.fit(X, Y, epochs=20, batch_size=2048, callbacks=callbacks_list)

In [252]:
# export_results(history, ['loss'], image_only=False)
# export_results(history, ['categorical_accuracy'], image_only=False)

# Preprocessing for alternative model B

In [253]:
X, Y = init_sequences_b(corpus, char_to_idx, T_x=n_time_steps)
X

<tf.Tensor: shape=(2248, 100), dtype=int64, numpy=
array([[25,  2, 43, ..., 29,  2, 43],
       [29, 40,  2, ..., 36, 39, 43],
       [40, 32, 49, ..., 25, 44,  2],
       ...,
       [39,  2, 30, ..., 32, 33, 43],
       [44, 32, 29, ...,  2, 26, 42],
       [25, 35, 43, ...,  0,  0,  0]], dtype=int64)>

In [254]:
len(X)

2248

In [255]:
len(X[-2])

100

In [256]:
len(X[-1])

100

In [257]:
len(Y[-1])

100

In [258]:
X[-1]

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([25, 35, 43,  2, 37, 49,  2, 26, 29, 36, 33, 29, 30,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int64)>

# Load a saved model
* see arhictecure
* see if prediction will work properly on dummy data

In [259]:
saved_model = load_model('./saved/models/test_model_b.h5')

In [260]:
saved_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 X (InputLayer)              [(None, 50)]                 0         []                            
                                                                                                  
 character-lookup (Embeddin  (None, 50, 32)               1824      ['X[0][0]']                   
 g)                                                                                               
                                                                                                  
 tf.__operators__.getitem (  (None, 32)                   0         ['character-lookup[0][0]']    
 SlicingOpLambda)                                                                                 
                                                                                              

#### Recall that our model needs 3 inputs, X, the hidden state, and the cell state. Because we are generating novel sequences using our trained model we pass in a $(1, 100)$ input where it represents the shape $(m, T_x)$, moreover our hidden and cell states remain the same in terms of their shape which is $(m, n_a)$ but only now it would be $(1, n_a)$ since we are passing only one input example to our model

In [261]:
sample_input = tf.random.uniform(shape=(1, 50), minval=0, maxval=n_unique - 1, dtype=tf.int32)

sample_h = tf.zeros(shape=(1, 128))
sample_c = tf.zeros(shape=(1, 128))

#### use the model to predict an output Y which we know will be of shape $(T_y, m, n_{unique})$ or in this case since we only inputted one example $(T_y, 1, 26)$

In [262]:
saved_model.predict([sample_input, sample_h, sample_c])

1/1 [==============================] - 23s 23s/step


[array([[  1.6799104 ,   3.8669317 ,   2.209147  ,   4.6212764 ,
          -8.245788  ,   0.05997242,  -0.4308589 ,  -5.1176095 ,
          -2.015434  ,   0.12388352,  -7.4533677 ,  -5.117378  ,
           8.06772   ,   2.754525  ,   3.3055038 ,   0.7376344 ,
          -3.4898863 ,   1.906194  ,  -2.6332285 ,  -5.289338  ,
           1.6305017 ,  -6.069223  , -10.386613  ,   4.597201  ,
          -3.7343388 ,  -7.3355694 ,   4.65129   ,  13.986267  ,
           7.134068  ,  -2.891338  ,   3.7720668 ,  14.603409  ,
          -5.360661  ,  -3.366272  ,  -0.22024786,  -1.0379428 ,
          -0.3783067 ,  11.032436  ,  -8.611395  ,  -5.0148807 ,
           1.8902985 ,  -4.435052  ,  -6.3586802 ,  -5.5456514 ,
          -6.4740305 ,   2.74524   ,  -0.17572576,  -0.47064108,
          -1.0018785 ,  -0.6783811 ,   3.8350613 ,  -0.8007593 ,
          -3.2351995 ,  -1.2378389 ,  -1.6098145 ,   1.250047  ,
           8.766432  ]], dtype=float32),
 array([[ 1.4404092 ,  3.7468374 ,  2.883557  ,  

#### List all layer names
* the goal here is to extract the Embedding, LSTM, Dense, and BatchNormalization layers used in training the model which have all been instantiated once and where we can extract and use as objects in our inference model
* Once we know the layers, we can start by training our model for training, and save the best model with the lowest loss value
* Access the saved models aforementioned layers and use it in the inference model

In [263]:
layers = saved_model.layers
for layer in layers:
    print(layer.name)

X
character-lookup
tf.__operators__.getitem
reshape-layer
init-hidden-state
init-cell-state
tf.__operators__.getitem_1
lstm-cell
tf.__operators__.getitem_2
tf.__operators__.getitem_3
tf.__operators__.getitem_4
tf.__operators__.getitem_5
tf.__operators__.getitem_6
tf.__operators__.getitem_7
tf.__operators__.getitem_8
tf.__operators__.getitem_9
tf.__operators__.getitem_10
tf.__operators__.getitem_11
tf.__operators__.getitem_12
tf.__operators__.getitem_13
tf.__operators__.getitem_14
tf.__operators__.getitem_15
tf.__operators__.getitem_16
tf.__operators__.getitem_17
tf.__operators__.getitem_18
tf.__operators__.getitem_19
tf.__operators__.getitem_20
tf.__operators__.getitem_21
tf.__operators__.getitem_22
tf.__operators__.getitem_23
tf.__operators__.getitem_24
tf.__operators__.getitem_25
tf.__operators__.getitem_26
tf.__operators__.getitem_27
tf.__operators__.getitem_28
tf.__operators__.getitem_29
tf.__operators__.getitem_30
tf.__operators__.getitem_31
tf.__operators__.getitem_32
tf.__operat

In [264]:
lstm_cell = saved_model.get_layer('lstm-cell')
embedding_layer = saved_model.get_layer('character-lookup')
dense_layer = saved_model.get_layer('dense-layer')
norm_layer = saved_model.get_layer('norm-layer')

In [265]:
lstm_cell.get_weights()

[array([[-4.3476768e-02,  4.1303251e-02, -4.4148944e-02, ...,
          1.6288672e-01, -1.6537686e-01,  1.0875196e-04],
        [ 1.9597633e-02,  5.9624646e-02, -1.1834999e-02, ...,
         -3.9792225e-02,  3.7065607e-02, -2.2591623e-02],
        [-2.0588848e-01,  1.6710361e-02, -2.8141338e-02, ...,
          1.2914309e-02, -1.5956153e-01,  1.0704896e-01],
        ...,
        [-7.9540946e-02,  3.3093460e-02,  1.1581952e-02, ...,
         -4.5030676e-02, -2.1205565e-01, -1.6505544e-01],
        [ 3.3483360e-02,  2.1167366e-02, -1.1593581e-01, ...,
          1.3973629e-01, -1.3845742e-01,  6.7909345e-02],
        [ 1.4139892e-01,  1.6884925e-02,  5.3639859e-03, ...,
          1.3745481e-01, -8.7566793e-02, -1.3118197e-03]], dtype=float32),
 array([[ 7.4825041e-02,  9.2818089e-02,  5.4825697e-02, ...,
          9.2428669e-02,  7.4624792e-02,  3.9098896e-02],
        [ 1.3704700e-02,  3.3305481e-02,  7.1759403e-02, ...,
          8.8618197e-02, -3.8863264e-03,  6.5765738e-02],
        [ 

In [266]:
embedding_layer.get_weights()

[array([[-0.00384016,  0.00852905, -0.03279462, ..., -0.05673631,
          0.05147038, -0.05086876],
        [-0.02852797, -0.00917254, -0.03751275, ...,  0.07995425,
         -0.01136292, -0.07683186],
        [-0.03099143,  0.02598903, -0.05246466, ..., -0.03805547,
         -0.02549372, -0.05019166],
        ...,
        [-0.03841486, -0.02107484,  0.0773114 , ...,  0.02192144,
          0.01932819,  0.07190665],
        [ 0.12292158, -0.01019445,  0.02425437, ...,  0.05735749,
          0.04937045, -0.03999884],
        [ 0.09614896,  0.0608432 ,  0.02769189, ...,  0.07925435,
          0.09934697, -0.01334613]], dtype=float32)]

In [267]:
dense_layer.get_weights()

[array([[-0.08887742,  0.00141365,  0.2281212 , ..., -0.02308418,
          0.12859   ,  0.05321813],
        [-0.11829051, -0.04015279, -0.07933553, ..., -0.04232099,
          0.13974907, -0.04237451],
        [ 0.10663955, -0.11148715,  0.16611011, ..., -0.18678807,
         -0.2612285 ,  0.05973718],
        ...,
        [-0.1163478 , -0.00490158,  0.16170481, ..., -0.0319512 ,
          0.20927925,  0.06769408],
        [-0.13686435,  0.04136466, -0.07427052, ..., -0.08803699,
         -0.1299074 ,  0.22134577],
        [-0.14147282, -0.14518335, -0.16522515, ...,  0.11709297,
         -0.01494861, -0.19647491]], dtype=float32),
 array([ 0.00418801,  0.01214846, -0.01558098, -0.0090064 ,  0.01991371,
         0.00321615,  0.02495515, -0.01436132,  0.0015014 ,  0.00710467,
         0.00682143,  0.00091798,  0.03793066, -0.02810089,  0.00503928,
        -0.05084297,  0.00035308, -0.0155677 ,  0.01876972,  0.01833175,
        -0.02066987,  0.00832202,  0.00956238, -0.01571993, -0.002

In [268]:
norm_layer.get_weights()

[array([1.2975014, 1.3070111, 1.3313695, 1.3173883, 1.3328098, 1.2934463,
        1.3719115, 1.2807202, 1.386137 , 1.3162878, 1.3578088, 1.3297181,
        1.3232323, 1.3509167, 1.311294 , 1.354796 , 1.3324177, 1.33188  ,
        1.3310574, 1.2910997, 1.3203034, 1.3582269, 1.3000629, 1.3535869,
        1.3114046, 1.3761654, 1.3442494, 1.3283615, 1.3807832, 1.3326049,
        1.3443154, 1.3658302, 1.3607246, 1.3423982, 1.2884866, 1.3427917,
        1.3350304, 1.3778646, 1.3561076, 1.3819553, 1.307686 , 1.3453972,
        1.2838575, 1.3278008, 1.3249692, 1.3697062, 1.3075763, 1.3776946,
        1.3553677, 1.344777 , 1.3866744, 1.2834014, 1.2973676, 1.3232545,
        1.313788 , 1.3467782, 1.2746987], dtype=float32),
 array([ 0.09032989, -0.04052205, -0.1638363 , -0.04692467, -0.00024327,
         0.05752519,  0.02586935, -0.09189001,  0.03668069, -0.14177969,
         0.00307033, -0.09115244,  0.10120527, -0.01468554, -0.04760839,
         0.03615911,  0.00892458, -0.01983972, -0.1016268

# Load saved weights

In [269]:
saved_model = GenPhiloText(emb_dim=32, n_a=128, n_unique=57, T_x=50)
saved_model([sample_input, sample_h, sample_c])

[<tf.Tensor: shape=(1, 57), dtype=float32, numpy=
 array([[-6.2079047e-04,  1.2357634e-03,  4.0326379e-03, -1.1000568e-03,
         -3.8865497e-03,  2.1002765e-03, -4.0199794e-03,  3.9545181e-03,
          4.1366033e-03,  3.9018819e-03, -3.4582303e-05, -1.0170757e-03,
          5.9963311e-03, -3.0224130e-04,  2.0506054e-03, -1.4629010e-03,
         -1.7685831e-03,  1.5412224e-03,  1.0536521e-03,  2.7615465e-03,
         -6.6799310e-04, -2.9965749e-03, -8.8691525e-04,  4.0799839e-04,
         -2.6788611e-03,  1.7322467e-03,  8.7116210e-04, -1.3137991e-03,
          3.9024578e-04, -4.3284688e-03, -3.1734095e-03, -1.3887333e-03,
         -3.2672120e-04, -5.4441057e-03, -3.7737812e-03,  7.4457566e-05,
          1.5208032e-03,  3.1105317e-03, -3.1777890e-03,  5.3885402e-03,
         -2.8745271e-04, -4.4830912e-03, -2.1525432e-04, -5.9885794e-04,
          2.8745760e-03, -7.0699956e-04, -1.7588950e-03,  2.8802713e-03,
          3.4695908e-03, -8.1180729e-04,  1.5025064e-03, -2.0910476e-03,
 

In [270]:
saved_model.summary()

Model: "gen_philo_text_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 char-emb-layer (Embedding)  multiple                  1824      
                                                                 
 lstm-cell (LSTM)            multiple                  82432     
                                                                 
 dense-layer (Dense)         multiple                  7353      
                                                                 
 activation_14 (Activation)  multiple                  0 (unused)
                                                                 
 reshape-layer (Reshape)     multiple                  0         
                                                                 
 norm-layer (BatchNormaliza  multiple                  228       
 tion)                                                           
                                                  

In [271]:
saved_model.load_weights(filepath='./saved/weights/test_model_gen_philo_text.h5')

In [272]:
saved_model.predict([sample_input, sample_h, sample_c])

1/1 [==============================] - 23s 23s/step


[array([[  5.390581  ,   0.86834884,  -8.168532  ,   3.3990357 ,
          -9.922525  ,   3.490132  ,  -3.5330157 ,  -4.249252  ,
          -0.3480667 ,  -7.310047  ,   7.4602976 ,  11.786769  ,
           9.05393   ,   6.285129  ,  -2.3434021 ,   9.424349  ,
           7.458408  ,  -4.0783515 , -13.995217  ,   0.82737696,
           5.444425  ,  -1.4200482 , -15.703529  , -17.690647  ,
          -7.734102  ,   4.7441735 ,  15.317133  ,   2.9867036 ,
           1.199963  ,  -0.82523125,   1.0534358 ,  -1.9567357 ,
           4.5612874 ,  -3.941423  ,   1.0856646 ,  -9.501804  ,
           7.5128303 , -11.632324  ,  -8.4340925 ,   1.8557324 ,
           3.2322845 ,  -7.9836636 ,  -0.24476936,  11.102614  ,
          13.598576  ,   7.1100364 ,  12.166502  , -13.29709   ,
          -1.3420906 ,   2.9774125 ,   5.5333595 ,  -9.617383  ,
          -2.1762958 ,   9.4535    ,  -3.65687   ,   1.6834574 ,
          -4.291757  ]], dtype=float32),
 array([[  5.3615794 ,   0.81877786,  -7.5357757 

In [273]:
layers = saved_model.layers
for layer in layers:
    print(layer.name)

char-emb-layer
lstm-cell
dense-layer
activation_14
reshape-layer
norm-layer


In [274]:
char_emb_layer = saved_model.get_layer('char-emb-layer')
char_emb_layer.get_weights()

[array([[ 0.05231718, -0.03042729, -0.00650564, ...,  0.03590847,
          0.0730855 ,  0.02147456],
        [ 0.09916682, -0.04843025,  0.03937633, ...,  0.02665502,
          0.1098014 ,  0.04500852],
        [ 0.04686486, -0.03721983, -0.09167735, ...,  0.08423789,
          0.00574228, -0.03700016],
        ...,
        [ 0.01303773, -0.01493781, -0.08189284, ..., -0.07163633,
          0.03615775, -0.00816663],
        [-0.02313188, -0.06718452, -0.03480035, ...,  0.02809362,
          0.03535094, -0.11069226],
        [-0.05088222, -0.07396545,  0.03798797, ..., -0.07236572,
         -0.02450815,  0.00088602]], dtype=float32)]

In [275]:
lstm_cell = saved_model.get_layer('lstm-cell')
lstm_cell.get_weights()

[array([[ 6.1332129e-02, -1.7190455e-01, -1.3345775e-01, ...,
          4.3923471e-02, -5.4484289e-03, -7.7299997e-02],
        [-5.8875438e-02, -1.0617943e-01, -6.4353377e-02, ...,
          4.4932500e-02, -7.2010525e-02, -1.4131120e-01],
        [ 1.9433571e-01, -1.4796652e-01, -5.4478824e-02, ...,
         -1.0230311e-01,  1.7290218e-01, -8.8213049e-02],
        ...,
        [-8.5743293e-03, -5.1982466e-02,  8.4154651e-02, ...,
         -4.0124949e-02, -1.3943009e-01, -2.0504977e-01],
        [-3.3181068e-02,  7.5101472e-02,  1.0956988e-02, ...,
          2.6174510e-01,  1.5926778e-01,  1.7892781e-01],
        [-4.9177013e-02,  1.7554225e-01, -2.2889557e-04, ...,
          1.9012861e-01,  1.0262496e-01, -4.6631187e-02]], dtype=float32),
 array([[ 0.02077633, -0.02451222, -0.01433547, ..., -0.01137133,
          0.05979912, -0.07633075],
        [-0.00663182, -0.14868075,  0.06071443, ..., -0.05265077,
         -0.1480431 , -0.18074183],
        [-0.04936531, -0.07722074, -0.05044382

In [276]:
dense_layer = saved_model.get_layer('dense-layer')
dense_layer.get_weights()

[array([[-0.1386417 , -0.09907272,  0.11679766, ...,  0.07776435,
          0.17212433, -0.23600106],
        [ 0.13277696, -0.09490051, -0.096317  , ..., -0.02153254,
          0.04380482, -0.03953426],
        [ 0.15760246,  0.09414165, -0.23709837, ..., -0.08401768,
         -0.24938457, -0.10208735],
        ...,
        [ 0.08797004,  0.23703073, -0.08753729, ..., -0.11081879,
          0.1849233 ,  0.09377686],
        [-0.08848459, -0.22668046, -0.11491586, ...,  0.11161321,
         -0.13195696,  0.11604118],
        [-0.16470592, -0.0192517 , -0.11872626, ..., -0.01735375,
         -0.07755947,  0.19025259]], dtype=float32),
 array([-1.9022973e-02, -2.0872377e-02, -1.6908510e-02, -4.3891378e-02,
        -6.3684634e-03,  5.2189627e-03, -6.9876118e-03,  2.3217106e-02,
        -6.9174189e-03,  1.5973406e-02,  1.0650595e-02, -3.2555446e-02,
         7.1959319e-03, -9.6036578e-03, -2.4816651e-02, -1.4396435e-02,
        -1.6645601e-02, -8.7271500e-03, -2.8792653e-02, -3.0236321e-03

In [277]:
norm_layer = saved_model.get_layer('norm-layer')
norm_layer.get_weights()

[array([1.2759098, 1.3029288, 1.2862264, 1.3314381, 1.3036861, 1.3609562,
        1.3334569, 1.3174341, 1.314854 , 1.3229043, 1.2872461, 1.3439531,
        1.2721273, 1.3094903, 1.3763553, 1.3772379, 1.3651625, 1.3225011,
        1.3804636, 1.3488654, 1.338196 , 1.3242769, 1.3498933, 1.3405616,
        1.3450181, 1.3503292, 1.3438351, 1.2774316, 1.2992827, 1.2745056,
        1.3716384, 1.293004 , 1.3161305, 1.3632063, 1.3154842, 1.314177 ,
        1.2887142, 1.3134595, 1.377387 , 1.31929  , 1.3534263, 1.335229 ,
        1.3463329, 1.3218738, 1.3229133, 1.3460915, 1.3046216, 1.3655772,
        1.335445 , 1.3007684, 1.2768046, 1.3477699, 1.2930162, 1.3101311,
        1.3456913, 1.3299284, 1.3564622], dtype=float32),
 array([ 0.05446801,  0.16916545,  0.01454173, -0.15858912, -0.05343375,
        -0.03034148,  0.06900607, -0.11306273,  0.00521365, -0.00693115,
        -0.0461528 , -0.10294751, -0.05826495,  0.0118617 ,  0.0581707 ,
         0.12654454, -0.06418873,  0.0928257 , -0.0830702

In [278]:
inference_model = load_inf_model(char_emb_layer, lstm_cell, dense_layer, norm_layer, char_to_idx)

57
128
tf.Tensor(
[[-inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]], shape=(1, 57), dtype=float32)


In [279]:
inference_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_14 (InputLayer)       [(None, 1)]                  0         []                            
                                                                                                  
 char-emb-layer (Embedding)  (None, 1, 32)                1824      ['input_14[0][0]',            
                                                                     'reshape_4[0][0]',           
                                                                     'reshape_4[1][0]',           
                                                                     'reshape_4[2][0]',           
                                                                     'reshape_4[3][0]',           
                                                                     'reshape_4[4][0]',     

In [280]:
sample_input = tf.random.uniform(shape=(1, 1), minval=0, maxval=n_unique - 1, dtype=tf.int32)

sample_h = tf.zeros(shape=(1, 128))
sample_c = tf.zeros(shape=(1, 128))

In [302]:
sample_input

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[48]])>

In [282]:
pred_ids = inference_model.predict([sample_input, sample_h, sample_c])

1/1 [==============================] - 0s 49ms/step


In [284]:
pred_ids

[array([[26]], dtype=int64),
 array([[26]], dtype=int64),
 array([[26]], dtype=int64),
 array([[26]], dtype=int64),
 array([[26]], dtype=int64),
 array([[26]], dtype=int64),
 array([[26]], dtype=int64),
 array([[26]], dtype=int64),
 array([[26]], dtype=int64),
 array([[26]], dtype=int64),
 array([[26]], dtype=int64),
 array([[26]], dtype=int64),
 array([[26]], dtype=int64),
 array([[26]], dtype=int64),
 array([[11]], dtype=int64),
 array([[11]], dtype=int64),
 array([[11]], dtype=int64),
 array([[11]], dtype=int64),
 array([[24]], dtype=int64),
 array([[11]], dtype=int64),
 array([[24]], dtype=int64),
 array([[24]], dtype=int64),
 array([[24]], dtype=int64),
 array([[24]], dtype=int64),
 array([[24]], dtype=int64),
 array([[23]], dtype=int64),
 array([[23]], dtype=int64),
 array([[23]], dtype=int64),
 array([[23]], dtype=int64),
 array([[23]], dtype=int64),
 array([[23]], dtype=int64),
 array([[23]], dtype=int64),
 array([[23]], dtype=int64),
 array([[23]], dtype=int64),
 array([[23]],

In [305]:
decode_predictions(pred_ids, idx_to_char)

"bbbbbbbbbbbbbb....?.?????;;;;;;;;;;´……ww'66666666666666666...................´´´´bbbb888888eee………………"